In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import ompy as om
import logging
import pymc3 as pm
import arviz as az

In [3]:
om.__full_version__

'1.1.0.dev0+02f42f6'

# Global settings of notebook
In the cell below there are a nuber of global settings that will affect the notebook from here. This is to make it easier to do a short PoC without having it to run for hours and hours.

* `ensemble_size` controls the number of ensemble members that gets generated. Depending on the operation done the runtime may be linear or N^(N-1).
* `multinest_livepoints` controls the number of live points used by PyMultinest. It is recomended to use at least 400 when accuracy is important. For debugging purposes this can be lowered.

In [4]:
ensemble_size = 10
multinest_livepoints = 300

In [5]:
def draw_random(matrix: om.Matrix, count: int, **kwargs) -> om.Matrix:
    """ Simple function to generate a matrix with N counts
    where the probability of drawing a particular bin is given
    by the number of counts in the input matrix.
    Args:
        matrix: (Matrix) Matrix giving the probability
        count: (int) Number of counts to draw
        **kwargs: Keyword arguments to the random number generator.
    Returns:
        A matrix-type with bin contents randomly drawn with the
        same distribution as the input matrix.
    """
    
    # Step 1, make a giant array with number of entries equal to the number of counts in each bin in the matrix.
    coords = []
    probability = []
    for i in range(len(matrix.Ex)):
        for j in range(len(matrix.Eg)):
            coords.append([i,j])
            probability.append(int(matrix.values[i,j]))
    
    coords = np.array(coords)
    probability = np.array(probability, dtype=float)
    probability /= np.sum(probability)
    
    # Step 2, draw 'draws' number of elements from the list
    rng = np.random.default_rng(**kwargs)
    drawed = rng.choice(coords, size=int(count), p=probability)
    mat = matrix.copy()
    mat.values *= 0
    for i,j in drawed:
        mat.values[i,j] += 1
    return mat

# Loading data
In this notebook we will be looking at the same data-set as in the [Getting Started](https://ompy.readthedocs.io/en/latest/getting_started.html) tutorial, but will do the analysis with three different statistics levels. To simulate low statistics we will use the `draw_random` function declared above. We will be looking at the full statistics case (5,279,152 counts), a low count case (100,000 counts) and a medium count case (250,000 counts).

The $^{164}\mathrm{Dy}$ data used below has been gathered from following experiment: Nyhus, H. T. *et al.* (2010). DOI: [10.1103/physrevc.81.024325](https://doi.org/10.1103/PhysRevC.81.024325)
and is reanalyzed in Renstrøm, T. *et al.* (2018). DOI: [10.1103/physrevc.98.054310](https://doi.org/10.1103/PhysRevC.98.054310)

In [6]:
# Import raw matrix into instance of om.Matrix() and plot it
raw = om.example_raw('Dy164')
# To use you own data, uncomment/adapt the line below instead
# raw = om.Matrix(path="/path/to/matrix.ending")

print(f"Total number of counts in raw matrix: {np.sum(raw.values)}")

# Cut the diagonal
raw.cut_diagonal(E1=(800, 0), E2=(7500, 7300))
raw.cut('Ex', 0, 8400)

# Plot the entire matrix
raw_org = raw.copy() # workaround due to execution order in jupyter notebook
                     # (calculations are performed before plotting, but we make a cut to raw further down)
raw_org.plot();

Total number of counts in raw matrix: 5279151.998904223


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Declare and define used classes
We are declaring all our analysis classes in the cells bellow as we only need to do this once.

In [7]:
logger = om.introspection.get_logger('response', 'INFO')
# Then do the same using OMpy functionality:
# You may need to adpot this to whereever you response matrices are stored
folderpath = "../OCL_response_functions/oscar2017_scale1.15"

# Energy calibration of resulting response matrix:
Eg = raw.Eg

# Experimental relative FWHM at 1.33 MeV of resulting array
fwhm_abs = 30 # (30/1330 = 2.25% )

# Magne recommends 1/10 of the actual resolution for unfolding purposes
response = om.Response(folderpath)
R_ompy_unf, R_tab_unf = response.interpolate(Eg, fwhm_abs=fwhm_abs/10, return_table=True)

2021-08-12 09:56:07,851 - ompy.response - INFO - Note: Spectra outside of 200.0 and 20000.0 are extrapolation only.


In [8]:
# With compton subtraction and all tweaks
unfolder= om.Unfolder(response=R_ompy_unf)
firstgen = om.FirstGeneration()
unfolder.use_compton_subtraction = True # default
unfolder.response_tab = R_tab_unf
# Magne suggests some "tweaks" for a better unfolding performance. Default is 1 for all.
unfolder.FWHM_tweak_multiplier = {"fe": 1., "se": 1.1,
                                     "de": 1.3, "511": 0.9}

trapezoid_cut = om.Action('matrix')
trapezoid_cut.trapezoid(Ex_min=4000, Ex_max=7000, Eg_min=1000, Eg_max=7000+200, inplace=True)
E_rebinned = np.arange(100., 8500, 200)

### Normalization parameters
See [getting started]() for more details.

In [9]:
norm_pars = om.NormalizationParameters(name="164Dy")
norm_pars.D0 = [6.8, 0.6]  # eV
norm_pars.Sn = [7.658, 0.001] # MeV
norm_pars.Gg = [113., 13.]  #meV
norm_pars.spincutModel = 'Disc_and_EB05'  # see eg. Guttormsen et al., 2017, PRC 96, 024313
norm_pars.spincutPars = {"mass":164, "NLDa":18.12, "Eshift":0.31,
                         "Sn": norm_pars.Sn[0], "sigma2_disc":[1.5, 3.6]}
norm_pars.Jtarget = 5/2 # A-1 nucleus

### Setup useful loggers

In [10]:
nld_log = om.introspection.get_logger('normalizer_nld', 'INFO')
gsf_log = om.introspection.get_logger('normalizer_gsf', 'INFO')
sim_log = om.introspection.get_logger('normalizer_simultan', 'INFO')
ens_log = om.introspection.get_logger('ensembleNormalizer', 'INFO')
logger = om.introspection.get_logger('error_finder', 'DEBUG')

## Declare ErrorFinder
This declares the error estimator using the logarithmic model. This is the more stable model.

In [11]:
error_estimator = om.error_finder.ErrorFinder(algorithm='log')

## Test dependence on statistics
In the next few cells we will generate input matrices with various number of counts. This is to simulate the effect of statistics has on the results.

In [12]:
raw_orig = raw.copy()
raw_orig.fill_and_remove_negative(window_size=2)

counts = [25000, 50000, 75000, 100000, 150000, 200000, 250000, 500000, 750000, 1000000, 1500000, 2000000]
raw_matrices = [draw_random(raw_orig, count) for count in counts]
raw_matrices.append(raw_orig.copy())
counts.append(int(raw_orig.counts))

### Setup the ensembles

In [13]:
ensembles = [om.Ensemble(raw=raw) for raw in raw_matrices]
for ensemble in ensembles:
    ensemble.unfolder = unfolder
    ensemble.first_generation_method = firstgen
    ensemble.generate(ensemble_size, regenerate=True)
    ensemble.rebin(E_rebinned, member="firstgen")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

### Setup extractors

In [15]:
extractors = [om.Extractor() for raw in raw_matrices]
for extractor, ensemble in zip(extractors, ensembles):
    extractor.trapezoid = trapezoid_cut
    extractor.extract_from(ensemble, regenerate=True)

  0%|          | 0/10 [00:00<?, ?it/s]

nld #0 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #1 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #2 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #3 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #4 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #5 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #6 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #7 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #8 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #9 contains nan's.
Consider removing them 

  0%|          | 0/10 [00:00<?, ?it/s]

nld #0 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #1 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #2 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #3 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #4 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #5 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #6 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #7 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #8 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #9 contains nan's.
Consider removing them 

  0%|          | 0/10 [00:00<?, ?it/s]

...

nld #0 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #1 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #2 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #3 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #4 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #5 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #6 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #7 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #8 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #9 contains nan's.
Consider removing them 

  0%|          | 0/10 [00:00<?, ?it/s]

nld #0 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #1 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #2 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #3 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #4 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #5 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #6 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #7 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #8 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #9 contains nan's.
Consider removing them 

  0%|          | 0/10 [00:00<?, ?it/s]

nld #0 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #1 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #2 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #3 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #4 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #5 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #6 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #7 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #8 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #9 contains nan's.
Consider removing them 

### Estimate relative uncertanties

In [17]:
nld_errors = []
gsf_errors = []
for extractor in extractors:
    nld_err, gsf_err = error_estimator.evaluate(extractor.nld, extractor.gsf)
    nld_errors.append(nld_err.copy())
    gsf_errors.append(gsf_err.copy())

2021-08-12 10:15:42,736 - ompy.error_finder - DEBUG - Processing an ensemble with 10 members
2021-08-12 10:15:42,737 - ompy.error_finder - DEBUG - Before removing nan: 45 NLD values and 31 GSF values
2021-08-12 10:15:42,739 - ompy.error_finder - WARNING - NLDs and/or γSFs contains nan's. They will be removed
2021-08-12 10:15:42,740 - ompy.error_finder - WARNING - Some members of the ensemble have different lengths. Consider re-binning or changing limits.
2021-08-12 10:15:42,743 - ompy.error_finder - DEBUG - After removing nan: 30 NLD values and 27 GSF values
2021-08-12 10:15:42,745 - ompy.error_finder - INFO - Starting pyMC3 inference - logarithmic model


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [σ_f, σ_ρ, α, F, D, σ_α, σ_F, σ_D]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 76 seconds.


2021-08-12 10:17:25,444 - ompy.error_finder - INFO - Inference results:
┌─────────────┬─────────────┬────────────────┬───────────────┬───────────────┬─────────────────┐
│ D           │ F           │ α              │ σ_D           │ σ_F           │ σ_α             │
╞═════════════╪═════════════╪════════════════╪═══════════════╪═══════════════╪═════════════════╡
│ 0.00 ± 0.27 │ 0.00 ± 0.17 │ -0.000 ± 0.049 │ 0.276 ± 0.022 │ 0.168 ± 0.015 │ 0.0492 ± 0.0043 │
└─────────────┴─────────────┴────────────────┴───────────────┴───────────────┴─────────────────┘
┌────┬──────────────┬──────────────┐
│    │ σ_ρ [%]      │ σ_f [%]      │
╞════╪══════════════╪══════════════╡
│ 0  │ 37.0 ± 2.9   │ 13.2 ± 1.1   │
├────┼──────────────┼──────────────┤
│ 1  │ 32.5 ± 2.5   │ 11.39 ± 0.98 │
├────┼──────────────┼──────────────┤
│ 2  │ 46.5 ± 3.7   │ 10.11 ± 0.87 │
├────┼──────────────┼──────────────┤
│ 3  │ 20.6 ± 1.7   │ 15.1 ± 1.2   │
├────┼──────────────┼──────────────┤
│ 4  │ 18.0 ± 1.5   │ 9.25 ± 0.78  │

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [σ_f, σ_ρ, α, F, D, σ_α, σ_F, σ_D]


...

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 94 seconds.


2021-08-12 10:35:22,537 - ompy.error_finder - INFO - Inference results:
┌──────────────┬──────────────┬───────────────┬───────────────┬───────────────┬─────────────────┐
│ D            │ F            │ α             │ σ_D           │ σ_F           │ σ_α             │
╞══════════════╪══════════════╪═══════════════╪═══════════════╪═══════════════╪═════════════════╡
│ -0.00 ± 0.26 │ -0.00 ± 0.26 │ 0.000 ± 0.064 │ 0.264 ± 0.020 │ 0.264 ± 0.020 │ 0.0646 ± 0.0049 │
└──────────────┴──────────────┴───────────────┴───────────────┴───────────────┴─────────────────┘
┌────┬──────────────┬──────────────┐
│    │ σ_ρ [%]      │ σ_f [%]      │
╞════╪══════════════╪══════════════╡
│ 0  │ 2.34 ± 0.26  │ 3.96 ± 0.33  │
├────┼──────────────┼──────────────┤
│ 1  │ 3.93 ± 0.33  │ 5.10 ± 0.40  │
├────┼──────────────┼──────────────┤
│ 2  │ 5.71 ± 0.44  │ 3.59 ± 0.29  │
├────┼──────────────┼──────────────┤
│ 3  │ 5.92 ± 0.48  │ 3.35 ± 0.28  │
├────┼──────────────┼──────────────┤
│ 4  │ 5.09 ± 0.42  │ 2.78 ± 0.

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [σ_f, σ_ρ, α, F, D, σ_α, σ_F, σ_D]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 96 seconds.


2021-08-12 10:37:10,134 - ompy.error_finder - INFO - Inference results:
┌──────────────┬──────────────┬───────────────┬───────────────┬───────────────┬─────────────────┐
│ D            │ F            │ α             │ σ_D           │ σ_F           │ σ_α             │
╞══════════════╪══════════════╪═══════════════╪═══════════════╪═══════════════╪═════════════════╡
│ -0.00 ± 0.26 │ -0.00 ± 0.26 │ 0.000 ± 0.067 │ 0.267 ± 0.020 │ 0.263 ± 0.020 │ 0.0679 ± 0.0050 │
└──────────────┴──────────────┴───────────────┴───────────────┴───────────────┴─────────────────┘
┌────┬──────────────┬─────────────┐
│    │ σ_ρ [%]      │ σ_f [%]     │
╞════╪══════════════╪═════════════╡
│ 0  │ 1.93 ± 0.22  │ 4.11 ± 0.33 │
├────┼──────────────┼─────────────┤
│ 1  │ 1.93 ± 0.21  │ 4.09 ± 0.34 │
├────┼──────────────┼─────────────┤
│ 2  │ 3.32 ± 0.26  │ 3.58 ± 0.27 │
├────┼──────────────┼─────────────┤
│ 3  │ 4.63 ± 0.36  │ 4.97 ± 0.39 │
├────┼──────────────┼─────────────┤
│ 4  │ 3.59 ± 0.29  │ 1.15 ± 0.14 │
├────┼

In [41]:
def make_vec(nlds, counts, point):
    return om.Vector(E=np.array(counts)/1.0e6, values=np.array([nld.values[nld.index(point)] for nld in nlds]),
                     std=np.array([nld.std[nld.index(point)] for nld in nlds]))

extractors[-1].plot()

err = nld_errors[0].copy()
err.values *= 100.
fig, ax = err.plot(label="N=25000")
for i in range(1, len(counts), 4):
    err = nld_errors[i].copy()
    err.values *= 100.
    err.plot(ax=ax, label=f"N={counts[i]}")

err = nld_errors[-1].copy()
err.values *= 100.
err.plot(ax=ax, label=f"N={counts[-1]}")
    
ax.set_ylim(0, 100)
ax.set_ylabel("Relative error [%]")
ax.legend(loc='best')

nld_0MeV = make_vec(nld_errors, counts, 0)
nld_15MeV = make_vec(nld_errors, counts, 1.5)
nld_3MeV = make_vec(nld_errors, counts, 3)
nld_5MeV = make_vec(nld_errors, counts, 5)

nld_0MeV = make_vec(nld_errors, counts, 0)
nld_0MeV.values *= 100.
nld_0MeV.std *= 100.
# nld_0MeV.save("nld_0MeV.csv", sep="\t")
nld_2MeV = make_vec(nld_errors, counts, 2)
nld_2MeV.values *= 100.
nld_2MeV.std *= 100.
# nld_2MeV.save("nld_2MeV.csv", sep="\t")
nld_4MeV = make_vec(nld_errors, counts, 4)
nld_4MeV.values *= 100.
nld_4MeV.std *= 100.
# nld_4MeV.save("nld_4MeV.csv", sep="\t")

nld_1MeV = make_vec(nld_errors, counts, 0)
_, ax = nld_0MeV.plot(label=r"$E_x = 0$ MeV")
nld_2MeV.plot(ax=ax, label=r"$E_x = 2$ MeV")
nld_4MeV.plot(ax=ax, label=r"$E_x = 4$ MeV")
ax.set_xlabel("Counts [$10^6$]")
c = np.linspace(min(counts), max(counts), 1001)
ax.plot(c/1e6, (np.sqrt(counts[0])*20.)*1./np.sqrt(c))
ax.legend(loc='best')
ax.set_ylabel("Relative uncertanty [%]");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Investigate dependence on ensemble members

In [19]:
ensemble_full = om.Ensemble(raw=raw_orig, path="saved_run/ensemble/full") # Note raw will be changed later manually.
ensemble_full.unfolder = unfolder
ensemble_full.first_generation_method = firstgen

extractors_ens = []
for n in range(3, 15):
    ensemble_full.generate(n, regenerate=True)
    ensemble_full.rebin(E_rebinned, member="firstgen")
    extractors_ens.append(om.Extractor())
    extractors_ens[-1].trapezoid = trapezoid_cut
    extractors_ens[-1].extract_from(ensemble_full, regenerate=True)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

nld #0 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #1 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #2 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

gsf #0 contains nan's.
Consider removing them e.g. with:
# for gsf in extractor.gsf:
#     gsf.cut_nan()

gsf #1 contains nan's.
Consider removing them e.g. with:
# for gsf in extractor.gsf:
#     gsf.cut_nan()

gsf #2 contains nan's.
Consider removing them e.g. with:
# for gsf in extractor.gsf:
#     gsf.cut_nan()



  0%|          | 0/4 [00:00<?, ?it/s]

...

nld #0 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #1 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #2 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #3 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #4 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #5 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #6 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #7 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #8 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #9 contains nan's.
Consider removing them 

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

nld #0 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #1 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #2 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #3 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #4 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #5 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #6 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #7 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #8 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #9 contains nan's.
Consider removing them 

In [20]:
nld_ens_errors = []
gsf_ens_errors = []
for extractor in extractors_ens:
    nld_err, gsf_err = error_estimator.evaluate(extractor.nld, extractor.gsf)
    nld_ens_errors.append(nld_err.copy())
    gsf_ens_errors.append(gsf_err.copy())

2021-08-12 10:43:04,405 - ompy.error_finder - DEBUG - Processing an ensemble with 3 members
2021-08-12 10:43:04,406 - ompy.error_finder - DEBUG - Before removing nan: 45 NLD values and 31 GSF values
2021-08-12 10:43:04,407 - ompy.error_finder - WARNING - NLDs and/or γSFs contains nan's. They will be removed
2021-08-12 10:43:04,408 - ompy.error_finder - DEBUG - After removing nan: 30 NLD values and 30 GSF values
2021-08-12 10:43:04,409 - ompy.error_finder - INFO - Starting pyMC3 inference - logarithmic model


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [σ_f, σ_ρ, α, F, D, σ_α, σ_F, σ_D]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 82 seconds.
There were 310 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.5323431089331814, but should be close to 0.8. Try to increase the number of tuning steps.
There were 29 divergences after tuning. Increase `target_accept` or reparameterize.
There were 117 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6415360461552452, but should be close to 0.8. Try to increase the number of tuning steps.
There were 44 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


2021-08-12 10:44:36,679 - ompy.error_finder - INFO - Inference results:
┌─────────────┬─────────────┬────────────────┬─────────────┬─────────────┬───────────────┐
│ D           │ F           │ α              │ σ_D         │ σ_F         │ σ_α           │
╞═════════════╪═════════════╪════════════════╪═════════════╪═════════════╪═══════════════╡
│ 0.00 ± 0.31 │ 0.00 ± 0.32 │ -0.000 ± 0.069 │ 0.39 ± 0.15 │ 0.41 ± 0.16 │ 0.088 ± 0.037 │
└─────────────┴─────────────┴────────────────┴─────────────┴─────────────┴───────────────┘
┌────┬───────────────┬─────────────┐
│    │ σ_ρ [%]       │ σ_f [%]     │
╞════╪═══════════════╪═════════════╡
│ 0  │ 2.32 ± 0.94   │ 3.2 ± 1.3   │
├────┼───────────────┼─────────────┤
│ 1  │ 0.046 ± 0.039 │ 1.38 ± 0.58 │
├────┼───────────────┼─────────────┤
│ 2  │ 0.049 ± 0.044 │ 2.5 ± 1.1   │
├────┼───────────────┼─────────────┤
│ 3  │ 3.3 ± 1.2     │ 3.3 ± 1.3   │
├────┼───────────────┼─────────────┤
│ 4  │ 2.8 ± 1.0     │ 0.85 ± 0.49 │
├────┼───────────────┼───────

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [σ_f, σ_ρ, α, F, D, σ_α, σ_F, σ_D]


...

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 134 seconds.


2021-08-12 11:00:31,261 - ompy.error_finder - INFO - Inference results:
┌─────────────┬──────────────┬────────────────┬───────────────┬───────────────┬─────────────────┐
│ D           │ F            │ α              │ σ_D           │ σ_F           │ σ_α             │
╞═════════════╪══════════════╪════════════════╪═══════════════╪═══════════════╪═════════════════╡
│ 0.00 ± 0.25 │ -0.00 ± 0.26 │ -0.000 ± 0.062 │ 0.250 ± 0.014 │ 0.258 ± 0.015 │ 0.0623 ± 0.0035 │
└─────────────┴──────────────┴────────────────┴───────────────┴───────────────┴─────────────────┘
┌────┬──────────────┬──────────────┐
│    │ σ_ρ [%]      │ σ_f [%]      │
╞════╪══════════════╪══════════════╡
│ 0  │ 1.56 ± 0.15  │ 4.46 ± 0.26  │
├────┼──────────────┼──────────────┤
│ 1  │ 2.14 ± 0.16  │ 3.91 ± 0.24  │
├────┼──────────────┼──────────────┤
│ 2  │ 3.82 ± 0.24  │ 3.97 ± 0.24  │
├────┼──────────────┼──────────────┤
│ 3  │ 4.52 ± 0.28  │ 5.29 ± 0.31  │
├────┼──────────────┼──────────────┤
│ 4  │ 5.44 ± 0.33  │ 1.88 ± 0.

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [σ_f, σ_ρ, α, F, D, σ_α, σ_F, σ_D]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 172 seconds.


2021-08-12 11:03:35,101 - ompy.error_finder - INFO - Inference results:
┌─────────────┬─────────────┬────────────────┬───────────────┬───────────────┬─────────────────┐
│ D           │ F           │ α              │ σ_D           │ σ_F           │ σ_α             │
╞═════════════╪═════════════╪════════════════╪═══════════════╪═══════════════╪═════════════════╡
│ 0.00 ± 0.28 │ 0.00 ± 0.27 │ -0.000 ± 0.067 │ 0.277 ± 0.015 │ 0.277 ± 0.014 │ 0.0674 ± 0.0035 │
└─────────────┴─────────────┴────────────────┴───────────────┴───────────────┴─────────────────┘
┌────┬──────────────┬───────────────┐
│    │ σ_ρ [%]      │ σ_f [%]       │
╞════╪══════════════╪═══════════════╡
│ 0  │ 1.57 ± 0.14  │ 4.22 ± 0.23   │
├────┼──────────────┼───────────────┤
│ 1  │ 2.08 ± 0.15  │ 3.81 ± 0.21   │
├────┼──────────────┼───────────────┤
│ 2  │ 4.02 ± 0.23  │ 3.80 ± 0.21   │
├────┼──────────────┼───────────────┤
│ 3  │ 4.37 ± 0.24  │ 5.04 ± 0.27   │
├────┼──────────────┼───────────────┤
│ 4  │ 5.32 ± 0.29  │ 1.8

In [21]:
def make_vec(nlds, point, counts=np.array(range(3,15))):
    return om.Vector(E=np.array(counts),
                     values=np.array([nld.std[nld.index(point)] for nld in nlds])/np.array([nld.values[nld.index(point)] for nld in nlds]))

nld_0MeV = make_vec(nld_ens_errors, 1)
nld_0MeV.values *= 100.
#nld_0MeV.save("nld_ens1MeV.csv", sep="\t")
nld_2MeV = make_vec(nld_ens_errors, 3)
nld_2MeV.values *= 100.
#nld_2MeV.save("nld_ens3MeV.csv", sep="\t")
nld_4MeV = make_vec(nld_ens_errors, 5)
nld_4MeV.values *= 100.
#nld_4MeV.save("nld_ens5MeV.csv", sep="\t")

_, ax = nld_0MeV.plot(label=r"$E_x = 1$ MeV", linestyle='')
nld_2MeV.plot(ax=ax, label=r"$E_x = 3$ MeV", linestyle='')
nld_4MeV.plot(ax=ax, label=r"$E_x = 5$ MeV", linestyle='')
ax.plot(np.linspace(3, 14, 1001), 10.*np.sqrt(8.*7.)/np.sqrt(np.linspace(3, 14, 1001)*(np.linspace(3, 14, 1001)-1)))
ax.set_xlabel("Ensemble members")
ax.set_ylabel("Relative error [%]")
ax.legend(loc='best');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Checking if the uncertanties makes sense
The estimated uncertanties are for NLDs and $\gamma$SFs that are taken from a stocastic input. The relative error of the total number of counts ($\sqrt{N}/N$) found in the first generation matrix should give the lower limit for the estimated uncertanties.

For the NLD we are interested in the total number of counts along the diagonals $P(E_f+E_\gamma, E_\gamma)$, $E_\gamma \in [E_\gamma^{min}, E_\gamma^{max}]$. For the $\gamma$SF we are interested in the horizonal lines, $P(E_x, E_\gamma)$, $E_x \in [E_x^{min}, E_x^{max}]$.

In [51]:
for ensemble, gsf_err in zip(ensembles, gsf_errors):
    
    matrix = ensemble.get_raw(0).copy()
    for n in range(1, len(ensemble.raw_ensemble)):
        matrix.values += ensemble.get_raw(n).values
    matrix.values *= 1./len(ensemble.raw_ensemble)
    mat = matrix.copy()
    mat.rebin('both', E_rebinned, inplace=True)
    trapezoid_cut.act_on(mat)
    
    vals = np.zeros(len(fg_matrix.Eg))
    for i in mat.range_Eg:
        for j in mat.range_Ex:
            vals[i] += mat.values[j,i]
    gsf_counts = om.Vector(E=mat.Eg.copy(), values=vals, units='keV')
    gsf_counts.to_MeV()
    gsf_counts.values = 100./np.sqrt(gsf_counts.values)
    
    fig, ax = gsf_counts.plot()
    er = gsf_err.copy()
    er.values *= 100.
    er.plot(ax=ax)
    ax.set_ylabel("Relative error [%]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
# Now similar but for NLD
# First we will add together all the first generation matrices in the ensemble and take the average for each bin.
fg_matrix = ensemble_full.get_firstgen(0)
for i in range(1, len(ensemble_full.raw_ensemble)):
    fg_matrix.values += ensemble_full.get_firstgen(i).values

fg_matrix.values *= 1./len(ensemble_full.raw_ensemble) # There are 15 ensemble members

# Absolute worst would be the statistics from the raw matrix?
#fg_matrix = raw_orig.copy()
fg_matrix.rebin('both', E_rebinned, inplace=True)


# Next we cut
trapezoid_cut.act_on(fg_matrix)

Ef = nld_ens_errors[-1].E*1000.
Ef0 = Ef[0]
values = np.zeros(len(Ef))
for i, ef in enumerate(Ef):
    for j in fg_matrix.range_Eg:
        if ef + fg_matrix.Eg[j] < min(fg_matrix.Ex) or ef + fg_matrix.Eg[j] > max(fg_matrix.Ex):
            continue
        index_Ex = fg_matrix.index_Ex(ef + fg_matrix.Eg[j])
        values[i] += fg_matrix.values[index_Ex, j]
        #print(i, " ", index_Ex, " ", j, " ", fg_matrix.Ex[index_Ex], " ", fg_matrix.Eg[j], " ", fg_matrix.Ex[index_Ex]-fg_matrix.Eg[j])
print(100./np.sqrt(values[-1]))
nld_counts = om.Vector(E=Ef, values=values, units='keV')
nld_counts.to_MeV()
nld_counts.values = 100./np.sqrt(nld_counts.values)
fig, ax = nld_counts.plot()
er = nld_ens_errors[-1].copy()
er.values *= 100.
er.plot(ax=ax)
ax.set_ylabel("Relative error [%]");

2.3833305269815916


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [96]:
for ensemble, nld_err in zip(ensembles, nld_errors):
    
    matrix = ensemble.get_raw(0).copy()
    for n in range(1, len(ensemble.raw_ensemble)):
        matrix.values += ensemble.get_raw(n).values
    matrix.values *= 1./len(ensemble.raw_ensemble)
    mat = matrix.copy()
    mat.rebin('both', E_rebinned, inplace=True)
    trapezoid_cut.act_on(mat)
        
    Ef = nld_err.E*1e3
    vals = np.zeros(len(Ef))
    for i, ef in enumerate(Ef):
        for j in fg_matrix.range_Eg:
            if ef + fg_matrix.Eg[j] < min(fg_matrix.Ex) or ef + fg_matrix.Eg[j] > max(fg_matrix.Ex):
                continue
            index_Ex = fg_matrix.index_Ex(ef + fg_matrix.Eg[j])
            vals[i] += fg_matrix.values[index_Ex, j]
    
    nld_counts = om.Vector(E=Ef, values=vals, units='keV')
    nld_counts.to_MeV()
    nld_counts.values = 100./np.sqrt(nld_counts.values)
    
    #fig, ax = nld_counts.plot()
    er = nld_err.copy()
    er.values *= 100.
    nld_counts.values *= er.values[15]/nld_counts.values[15]
    fig, ax = nld_counts.plot()
    er.plot(ax=ax)
    ax.set_ylabel("Relative error [%]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …